In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon

import matplotlib.pyplot as plt
import seaborn as sns

import os

import numpy as np
import os
import rasterio
import matplotlib.pyplot as plt
import pickle

In [ ]:
base_path = "/mnt/c/Users/laure/Data/SEN2-MSI-T/"
result_path = "/mnt/c/Users/laure/Data/results/performance_measures/"
result_path = "/mnt/c/Users/Laurens/Downloads/performance_measures/" # Main
#result_path = "/mnt/c/Users/Laurens/Downloads/performance_measures_l1c/" # NN comparison experiment


scenes = list(os.listdir(result_path))

features = ["1w", "1m", "3m", "6m"] 
#features = ["reconstruction"] # NN comparison experiment
#features = [" 1m"]
algs = ["VPint", "VPint_no_IP", "VPint_no_EB", "replacement", "regression", ]
        #"OracleEnsembleVPint", "OracleEnsembleNoVPint", "PatchOracleEnsembleVPint", "PatchOracleEnsembleNoVPint"]
#algs = ["VPint"]
#algs = ["VPint", "dsen2cr", "dsen2cr_nosar"] # NN comparison experiment

measure = "MAE"
CI_filter = False
CI_interval = 95

significance_level = 0.05

with open("invalid_patches.pkl", 'rb') as fp:
        invalid_patches = pickle.load(fp)

In [ ]:
# For visualising specific result patches

import numpy as np
import matplotlib.pyplot as plt

def normalise_and_visualise(img,title="",rgb=[3,2,1],percentile_clip=True,save_fig=False,**kwargs):
    
    new_img = np.zeros((img.shape[0],img.shape[1],3))
    new_img[:,:,0] = img[:,:,rgb[0]]
    new_img[:,:,1] = img[:,:,rgb[1]]
    new_img[:,:,2] = img[:,:,rgb[2]]
    
    if(percentile_clip):
        min_val = np.nanpercentile(new_img,1)
        max_val = np.nanpercentile(new_img,99)

        new_img = np.clip((new_img-min_val)/(max_val-min_val),0,1)
    
    plt.imshow(new_img,interpolation="nearest")
    plt.title(title)
    plt.axis('off')
    if(save_fig):
        plt.savefig(kwargs['path'],bbox_inches='tight')
    plt.show()


paths = ["/mnt/c/Users/laure/Downloads/grace/127.npy"]

for path in paths:
    a = np.load(path)
    normalise_and_visualise(a)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def normalise_and_visualise(img,title="",rgb=[3,2,1],percentile_clip=True,save_fig=False,**kwargs):
    
    new_img = np.zeros((img.shape[0],img.shape[1],3))
    new_img[:,:,0] = img[:,:,rgb[0]]
    new_img[:,:,1] = img[:,:,rgb[1]]
    new_img[:,:,2] = img[:,:,rgb[2]]
    
    if(percentile_clip):
        min_val = np.nanpercentile(new_img,1)
        max_val = np.nanpercentile(new_img,99)

        new_img = np.clip((new_img-min_val)/(max_val-min_val),0,1)
    
    plt.imshow(new_img,interpolation="nearest")
    plt.title(title)
    plt.axis('off')
    if(save_fig):
        plt.savefig(kwargs['path'],bbox_inches='tight')
    plt.show()


path = "/mnt/c/Users/laure/Downloads/grace/227.npy"
img = np.load(path)
normalise_and_visualise(img)

In [ ]:
# Visualising example patches per LC

base_path = "/mnt/c/Users/Laurens/Downloads/patch_vis/"
lcs = ["cropland", "forest", "herbaceous", "shrubs", "urban"]

def normalise_and_visualise(img,title="",rgb=[3,2,1],percentile_clip=True,save_fig=False,**kwargs):
    
    new_img = np.zeros((img.shape[0],img.shape[1],3))
    new_img[:,:,0] = img[:,:,rgb[0]]
    new_img[:,:,1] = img[:,:,rgb[1]]
    new_img[:,:,2] = img[:,:,rgb[2]]
    
    if(percentile_clip):
        min_val = np.nanpercentile(new_img,1)
        max_val = np.nanpercentile(new_img,99)

        new_img = np.clip((new_img-min_val)/(max_val-min_val),0,1)
    
    plt.imshow(new_img,interpolation="nearest")
    plt.title(title)
    plt.axis('off')
    if(save_fig):
        plt.savefig(kwargs['path'],bbox_inches='tight')
    plt.show()

for lc in lcs:
    patch = np.load(base_path + lc + ".npy")
    normalise_and_visualise(patch, save_fig=True, path="plots/patch_vis_"+lc+".pdf")


In [ ]:
# Check means per scene per feature

base_path = "/mnt/c/Users/laure/Data/SEN2-MSI-T/"
result_path = "/mnt/c/Users/laure/Data/results/performance_measures/"
result_path = "/mnt/c/Users/Laurens/Downloads/performance_measures/" # Main
result_path = "/mnt/c/Users/Laurens/Downloads/performance_measures_l1c/" # NN comparison experiment


scenes = list(os.listdir(result_path))

features = [" 1w", " 1m", " 3m"] # Messed up earlier so features have a leading space
features = ["reconstruction"] # NN comparison experiment
#features = [" 1m"]
algs = ["VPint", "VPint_no_IP", "VPint_no_EB", "replacement", "regression", 
        "OracleEnsembleVPint", "OracleEnsembleNoVPint", "PatchOracleEnsembleVPint", "PatchOracleEnsembleNoVPint"]
algs = ["VPint", "dsen2cr", "dsen2cr_nosar"] # NN comparison experiment
measure = "MAE"
CI_filter = True
CI_interval = 95

scenes = list(os.listdir(result_path))

show_local = True

total_vals = {}
for alg in algs:
    total_vals[alg] = np.array([])

for scene in scenes:
    path = result_path + scene + "/"
    if(show_local):
        print(scene)
    for feature in features:
        if(show_local):
            print(feature)
        for alg in algs:
            if(show_local):
                print(alg)
            path1 = path + alg + "_" + measure + ".csv"
            df = pd.read_csv(path1)
            df = df.drop_duplicates(subset=["patch"])
            vals = df[feature].values
            if(CI_filter):
                # Only keep results in specified confidence interval, since input data is not flawless
                min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                vals = vals[vals<max_val]
                vals = vals[vals>min_val]

            #vals = vals[vals < 5000] # TODO: remove, this is simulating filtering
            if(len(vals) > 0):
                if(show_local):
                    print(np.nanmean(vals), ", ", np.nanstd(vals))
                total_vals[alg] = np.concatenate((total_vals[alg], vals))
    if(show_local):
        print("\n")

print("Totals")
for alg in algs:
    print(alg)
    print(np.nanmean(total_vals[alg]), ", ", np.nanstd(total_vals[alg]))


In [ ]:
# Check means per land cover per feature

measure = "SSIM"

scenes = list(os.listdir(result_path))
lcs = ["cropland", "forest", "herbaceous", "shrubs", "urban"]

show_local = False

for lc in lcs:
    total_vals = {}
    for alg in algs:
        total_vals[alg] = np.array([])

    for scene in scenes:
        if(scene.split("_")[1] == lc):
            path = result_path + scene + "/"
            for feature in features:
                if(show_local):
                    print(feature)
                for alg in algs:
                    path1 = path + alg + "_" + measure + ".csv"
                    try:
                        df = pd.read_csv(path1)
                        df = df.drop_duplicates(subset=["patch"])
                        # Drop patches where coverage was not complete (e.g., due to swath)
                        invalid_patches_local = []
                        invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                        invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
                        df = df[~df['patch'].isin(invalid_patches_local)]
                        # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                        # so will not affect performance to remove these.
                        df = df[~df.patch.astype(str).str.isnumeric()] 
                        vals = df[feature].values.astype(float)
                        # Drop inf; we do this for MAPE, where fully black targets (caused by coverage
                        # issues) would result in infinite percentage errors
                        vals = vals[~np.isinf(vals)]
                        vals = vals[~np.isnan(vals)]
                        if(CI_filter):
                            # Only keep results in specified confidence interval, since input data is not flawless
                            min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                            max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                            vals = vals[vals<max_val]
                            vals = vals[vals>min_val]

                        if(len(vals) > 0):
                            if(show_local):
                                print(np.nanmean(vals), ", ", np.nanstd(vals))
                            total_vals[alg] = np.concatenate((total_vals[alg], vals))
                    except Exception as e:
                        print(e)
                        print(path)
                        dslkfjlskj

    print(lc)
    for alg in algs:
        print(alg)
        print(np.nanmean(total_vals[alg]), ", ", np.nanstd(total_vals[alg]))
    print("\n")


In [ ]:
# Wilcoxon test per land cover per feature


lcs = ["cropland", "forest", "herbaceous", "shrubs", "urban"]

measure = "SSIM"

show_local = False

for lc in lcs:
    total_vals = {}
    for alg in algs:
        total_vals[alg] = np.array([])

    for scene in scenes:
        if(scene.split("_")[1] == lc):
            path = result_path + scene + "/"
            for feature in features:
                if(show_local):
                    print(feature)
                for alg in algs:
                    path1 = path + alg + "_" + measure + ".csv"
                    try:
                        df = pd.read_csv(path1)
                        df = df.drop_duplicates(subset=["patch"])
                        # Drop patches where coverage was not complete (e.g., due to swath)
                        invalid_patches_local = []
                        invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                        invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
                        df = df[~df['patch'].isin(invalid_patches_local)]
                        # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                        # so will not affect performance to remove these.
                        df = df[~df.patch.astype(str).str.isnumeric()] 
                        vals = df[feature].values.astype(float)
                        # Drop inf; we do this for MAPE, where fully black targets (caused by coverage
                        # issues) would result in infinite percentage errors
                        vals = vals[~np.isinf(vals)]
                        vals = vals[~np.isnan(vals)]
                        if(CI_filter):
                            # Only keep results in specified confidence interval, since input data is not flawless
                            min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                            max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                            vals = vals[vals<max_val]
                            vals = vals[vals>min_val]

                        if(len(vals) > 0):
                            if(show_local):
                                print(np.nanmean(vals), ", ", np.nanstd(vals))
                            total_vals[alg] = np.concatenate((total_vals[alg], vals))
                    except Exception as e:
                        print(e)
                        print(path)
                        dslkfjlskj

    print(lc)
    # Compare every algorithm to every other algorithm; amount of significantly outperformed methods is rank (higher=better)
    num_outperformed = {}
    for alg in algs:
        num_outperformed[alg] = 0
        for alg2 in algs:
            vals1 = total_vals[alg]
            vals2 = total_vals[alg2]
            min_len = min(len(vals1), len(vals2))
            vals1 = vals1[:min_len]
            vals2 = vals2[:min_len]
            if(measure == "SSIM" or measure == "PSNR"):
                # Turn into minimisation
                vals1 = -vals1
                vals2 = -vals2
            if(np.mean(vals1) < np.mean(vals2)):
                w_test = wilcoxon(vals1, vals2, alternative="less")
                if(w_test[1] < significance_level):
                    num_outperformed[alg] = num_outperformed[alg] + 1
    print(num_outperformed)

    print("\n")


In [ ]:
# Check means per land cover per feature (ensembling)

measure = "MAE"

scenes = list(os.listdir(result_path))
lcs = ["cropland", "forest", "herbaceous", "shrubs", "urban"]

algs = ["OracleEnsembleVPint", "OracleEnsembleNoVPint", "PatchOracleEnsembleVPint", "PatchOracleEnsembleNoVPint"]

show_local = False

for lc in lcs:
    total_vals = {}
    for alg in algs:
        total_vals[alg] = np.array([])

    for scene in scenes:
        if(scene.split("_")[1] == lc):
            path = result_path + scene + "/"
            for feature in features:
                if(show_local):
                    print(feature)
                for alg in algs:
                    path1 = path + alg + "_" + measure + ".csv"
                    try:
                        df = pd.read_csv(path1)
                        df = df.drop_duplicates(subset=["patch"])
                        # Drop patches where coverage was not complete (e.g., due to swath)
                        invalid_patches_local = []
                        invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                        invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
                        df = df[~df['patch'].isin(invalid_patches_local)]
                        # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                        # so will not affect performance to remove these.
                        df = df[~df.patch.astype(str).str.isnumeric()] 
                        vals = df[feature].values.astype(float)
                        # Drop inf; we do this for MAPE, where fully black targets (caused by coverage
                        # issues) would result in infinite percentage errors
                        vals = vals[~np.isinf(vals)]
                        vals = vals[~np.isnan(vals)]
                        if(CI_filter):
                            # Only keep results in specified confidence interval, since input data is not flawless
                            min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                            max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                            vals = vals[vals<max_val]
                            vals = vals[vals>min_val]

                        if(len(vals) > 0):
                            if(show_local):
                                print(np.nanmean(vals), ", ", np.nanstd(vals))
                            total_vals[alg] = np.concatenate((total_vals[alg], vals))
                    except Exception as e:
                        print(e)
                        print(path)
                        dslkfjlskj

    print(lc)
    for alg in algs:
        print(alg)
        print(np.nanmean(total_vals[alg]), ", ", np.nanstd(total_vals[alg]))
    print("\n")


In [ ]:
# Wilcoxon test per land cover per feature (ensembling)


lcs = ["cropland", "forest", "herbaceous", "shrubs", "urban"]

measure = "MAE"

show_local = False

for lc in lcs:
    total_vals = {}
    for alg in algs:
        total_vals[alg] = np.array([])

    for scene in scenes:
        if(scene.split("_")[1] == lc):
            path = result_path + scene + "/"
            for feature in features:
                if(show_local):
                    print(feature)
                for alg in algs:
                    path1 = path + alg + "_" + measure + ".csv"
                    try:
                        df = pd.read_csv(path1)
                        df = df.drop_duplicates(subset=["patch"])
                        # Drop patches where coverage was not complete (e.g., due to swath)
                        invalid_patches_local = []
                        invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                        invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
                        df = df[~df['patch'].isin(invalid_patches_local)]
                        # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                        # so will not affect performance to remove these.
                        df = df[~df.patch.astype(str).str.isnumeric()] 
                        vals = df[feature].values.astype(float)
                        # Drop inf; we do this for MAPE, where fully black targets (caused by coverage
                        # issues) would result in infinite percentage errors
                        vals = vals[~np.isinf(vals)]
                        vals = vals[~np.isnan(vals)]
                        if(CI_filter):
                            # Only keep results in specified confidence interval, since input data is not flawless
                            min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                            max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                            vals = vals[vals<max_val]
                            vals = vals[vals>min_val]

                        if(len(vals) > 0):
                            if(show_local):
                                print(np.nanmean(vals), ", ", np.nanstd(vals))
                            total_vals[alg] = np.concatenate((total_vals[alg], vals))
                    except Exception as e:
                        print(e)
                        print(path)
                        dslkfjlskj

    print(lc)
    # Compare every algorithm to every other algorithm; amount of significantly outperformed methods is rank (higher=better)
    num_outperformed = {}
    for alg in algs:
        num_outperformed[alg] = 0
        for alg2 in algs:
            vals1 = total_vals[alg]
            vals2 = total_vals[alg2]
            min_len = min(len(vals1), len(vals2))
            vals1 = vals1[:min_len]
            vals2 = vals2[:min_len]
            if(measure == "SSIM" or measure == "PSNR"):
                # Turn into minimisation
                vals1 = -vals1
                vals2 = -vals2
            if(np.mean(vals1) < np.mean(vals2)):
                w_test = wilcoxon(vals1, vals2, alternative="less")
                if(w_test[1] < significance_level):
                    num_outperformed[alg] = num_outperformed[alg] + 1
    print(num_outperformed)

    print("\n")


In [ ]:
# Check means per land cover per feature (L1C)

measure = "SSIM"

lcs = ["cropland", "forest", "herbaceous", "shrubs", "urban"]

result_path_l1c = "/mnt/c/Users/Laurens/Downloads/performance_measures_l1c/"
scenes_l1c = list(os.listdir(result_path_l1c))

features = ["reconstruction"] 
algs = ["VPint", "dsen2cr", "dsen2cr_nosar"] 


show_local = False

for lc in lcs:
    total_vals = {}
    for alg in algs:
        total_vals[alg] = np.array([])

    for scene in scenes_l1c:
        if(scene.split("_")[1] == lc):
            path = result_path_l1c + scene + "/"
            for feature in features:
                if(show_local):
                    print(feature)
                for alg in algs:
                    path1 = path + alg + "_" + measure + ".csv"
                    try:
                        df = pd.read_csv(path1)
                        df = df.drop_duplicates(subset=["patch"])
                        # Drop patches where coverage was not complete (e.g., due to swath)
                        invalid_patches_local = []
                        #invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                        #invalid_patches_local.extend(invalid_patches[scene]['1m']['coverage'])
                        df = df[~df['patch'].isin(invalid_patches_local)]
                        # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                        # so will not affect performance to remove these.
                        df = df[~df.patch.astype(str).str.isnumeric()] 
                        vals = df[feature].values.astype(float)
                        # Drop inf; we do this for MAPE, where fully black targets (caused by coverage
                        # issues) would result in infinite percentage errors
                        vals = vals[~np.isinf(vals)]
                        vals = vals[~np.isnan(vals)]
                        if(CI_filter):
                            # Only keep results in specified confidence interval, since input data is not flawless
                            min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                            max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                            vals = vals[vals<max_val]
                            vals = vals[vals>min_val]

                        if(len(vals) > 0):
                            if(show_local):
                                print(np.nanmean(vals), ", ", np.nanstd(vals))
                            total_vals[alg] = np.concatenate((total_vals[alg], vals))
                    except Exception as e:
                        print(e)
                        print(path)
                        dslkfjlskj

    print(lc)
    for alg in algs:
        print(alg)
        print(np.nanmean(total_vals[alg]), ", ", np.nanstd(total_vals[alg]))
    print("\n")


In [ ]:
# Wilcoxon test per land cover per feature (L1C)


lcs = ["cropland", "forest", "herbaceous", "shrubs", "urban"]

result_path_l1c = "/mnt/c/Users/Laurens/Downloads/performance_measures_l1c/"
scenes_l1c = list(os.listdir(result_path_l1c))

features = ["reconstruction"] 
algs = ["VPint", "dsen2cr", "dsen2cr_nosar"] 

show_local = False

for lc in lcs:
    total_vals = {}
    for alg in algs:
        total_vals[alg] = np.array([])

    for scene in scenes_l1c:
        if(scene.split("_")[1] == lc):
            path = result_path_l1c + scene + "/"
            for feature in features:
                if(show_local):
                    print(feature)
                for alg in algs:
                    path1 = path + alg + "_" + measure + ".csv"
                    try:
                        df = pd.read_csv(path1)
                        df = df.drop_duplicates(subset=["patch"])
                        # Drop patches where coverage was not complete (e.g., due to swath)
                        invalid_patches_local = []
                        #invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                        #invalid_patches_local.extend(invalid_patches[scene]['1m']['coverage'])
                        df = df[~df['patch'].isin(invalid_patches_local)]
                        # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                        # so will not affect performance to remove these.
                        df = df[~df.patch.astype(str).str.isnumeric()] 
                        vals = df[feature].values.astype(float)
                        # Drop inf; we do this for MAPE, where fully black targets (caused by coverage
                        # issues) would result in infinite percentage errors
                        vals = vals[~np.isinf(vals)]
                        vals = vals[~np.isnan(vals)]
                        if(CI_filter):
                            # Only keep results in specified confidence interval, since input data is not flawless
                            min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                            max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                            vals = vals[vals<max_val]
                            vals = vals[vals>min_val]

                        if(len(vals) > 0):
                            if(show_local):
                                print(np.nanmean(vals), ", ", np.nanstd(vals))
                            total_vals[alg] = np.concatenate((total_vals[alg], vals))
                    except Exception as e:
                        print(e)
                        print(path)
                        dslkfjlskj

    print(lc)
    # Compare every algorithm to every other algorithm; amount of significantly outperformed methods is rank (higher=better)
    num_outperformed = {}
    for alg in algs:
        num_outperformed[alg] = 0
        for alg2 in algs:
            vals1 = total_vals[alg]
            vals2 = total_vals[alg2]
            min_len = min(len(vals1), len(vals2))
            vals1 = vals1[:min_len]
            vals2 = vals2[:min_len]
            if(measure == "SSIM" or measure == "PSNR"):
                # Turn into minimisation
                vals1 = -vals1
                vals2 = -vals2
            if(np.mean(vals1) < np.mean(vals2)):
                w_test = wilcoxon(vals1, vals2, alternative="less")
                if(w_test[1] < significance_level):
                    num_outperformed[alg] = num_outperformed[alg] + 1
    print(num_outperformed)

    print("\n")


In [ ]:
# Create boxplot per land cover class

import seaborn as sns


pretty_names_lc = {
    "urban":"Urban",
    "cropland":"Cropland",
    "herbaceous":"Herbaceous",
    "shrubs":"Shrubs",
    "forest":"Forest",
}
pretty_names_methods = {
    "VPint":"VPint2",
    "regression":"AutoML regression",
    "regression_band":"AutoML regression",
    "replacement":"Replacement"
}

algs = ["VPint", "VPint_no_IP", "VPint_no_EB", "replacement", "regression"]
algs = ["VPint", "regression", "replacement"]
lcs = ["cropland", "urban", "forest", "shrubs", "herbaceous"]
measure = "MAE"


scenes = os.listdir(result_path)

show_local = False

total_vals = {}
for alg in algs:
    total_vals[alg] = np.array([])

total_scenes = {}
for alg in algs:
    total_scenes[alg] = []

for scene in scenes:
    path = result_path + scene + "/"
    for feature in features:
        valset = []
        for alg in algs:
            path1 = path + alg + "_" + measure + ".csv"
            try:
                df = pd.read_csv(path1)
            except Exception as e:
                print(e) 
                print(path1)
                sldkfj
            df = df.drop_duplicates(subset=["patch"])
            # Drop patches where coverage was not complete (e.g., due to swath)
            invalid_patches_local = []
            invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
            invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
            df = df[~df['patch'].isin(invalid_patches_local)]
            # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
            # so will not affect performance to remove these.
            df = df[~df.patch.astype(str).str.isnumeric()] 
            vals = df[feature].values
            if(CI_filter):
                # Only keep results in specified confidence interval, since input data is not flawless
                min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                vals = vals[vals<max_val]
                vals = vals[vals>min_val]

            #vals = vals[vals < 5000] # TODO: remove, this is simulating filtering
            if(len(vals) > 0):
                total_vals[alg] = np.concatenate((total_vals[alg], vals))
                total_scenes[alg].extend([scene for _ in range(0, len(vals))])


total_vals_all = np.array([])
total_scenes_all = []
total_lcs_all = []
total_algs_all = []
for alg in algs:
    total_vals_all = np.concatenate((total_vals_all, total_vals[alg]))
    total_scenes_all.extend([a.split("_")[2] for a in total_scenes[alg]])
    total_lcs_all.extend([a.split("_")[1] for a in total_scenes[alg]])
    total_algs_all.extend([alg for _ in range(0, len(total_vals[alg]))])

total_algs_all = [pretty_names_methods[a] for a in total_algs_all]
df = pd.DataFrame({"Algorithm":total_algs_all, "Location":total_scenes_all, "Land cover":total_lcs_all,
                    measure:total_vals_all})

df['Land cover'] = df['Land cover'].apply(lambda x: pretty_names_lc[x])


#for lc in lcs:
#    df_local = df[df['Land cover'] == lc]
#plt.figure(figsize=(20,8))
p = sns.catplot(x='Land cover', y=measure, hue='Algorithm', kind='box', data=df)
p.fig.suptitle(measure + " per land cover class", y=1.02)
plt.ylim((0, 3500))
plt.savefig("plots/box_plot.pdf", bbox_inches='tight')
plt.show()


In [ ]:
# Create CD diagram (using autorank)

import autorank as ar
from autorank import autorank, plot_stats

scenes = list(os.listdir(result_path))

pretty_names_methods = {
    "VPint":"VPint2",
    "VPint_no_IP":"VPint2 (no IP)",
    "VPint_no_EB":"VPint2 (no EB)",
    "regression":"AutoML regression",
    "regression_band":"AutoML regression",
    "replacement":"Replacement",
}

#features = [" 1m"]
algs = ["VPint", "VPint_no_IP", "VPint_no_EB", "replacement", "regression_band"]
measures = ["MAE"]
significance_level = 0.05


for measure in measures:

    total_vals = {}
    for alg in algs:
        total_vals[pretty_names_methods[alg]] = np.array([])

    for scene in scenes:
        path = result_path + scene + "/"
        for feature in features:
            # First iterate over results per algorithm, store mean PM to be able to compute ranking
            alg_results = {}
            for alg in algs:
                path1 = path + alg + "_" + measure + ".csv"
                df = pd.read_csv(path1)
                df = df.drop_duplicates(subset=["patch"])
                # Drop patches where coverage was not complete (e.g., due to swath)
                invalid_patches_local = []
                invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                if(feature != "6m"):
                    invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
                df = df[~df['patch'].isin(invalid_patches_local)]
                # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                # so will not affect performance to remove these.
                df = df[~df.patch.astype(str).str.isnumeric()] 
                vals = df[feature].values.astype(float)
                vals = vals[~np.isnan(vals)]
                if(CI_filter):
                    # Only keep results in specified confidence interval, since input data is not flawless
                    min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                    max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                    vals = vals[vals<max_val]
                    vals = vals[vals>min_val]

                if(len(vals) > 0):

                    if(measure == "SSIM" or measure == "PSNR"):
                        # To easily compute ranking for the same objective
                        # Autorank automatically does maximisation, so flipping the negative in this cell
                        mean_val = np.mean(vals)
                    else:
                        mean_val = -np.mean(vals)

                    total_vals[pretty_names_methods[alg]] = np.concatenate((total_vals[pretty_names_methods[alg]], np.array([mean_val])))
    
                    
    # Draw CD diagram
    df = pd.DataFrame(total_vals)
    result = autorank(df, alpha=significance_level, verbose=False)
    #print(result)

    #plot_stats(result)
    ar._util.cd_diagram(result, reverse=False, ax=None, width=6) # Same as result but allowing reverse (1 on left)
    plt.savefig("plots/CD_" + measure + ".pdf", bbox_inches='tight')
    plt.show()


In [ ]:
# Create CD diagram (every patch a condition)

import autorank as ar
from autorank import autorank, plot_stats

scenes = list(os.listdir(result_path))

pretty_names_methods = {
    "VPint":"VPint2",
    "VPint_no_IP":"VPint2 (no IP)",
    "VPint_no_EB":"VPint2 (no EB)",
    "regression":"AutoML regression",
    "regression_band":"AutoML regression",
    "replacement":"Replacement",
}

#features = [" 1m"]
algs = ["VPint", "VPint_no_IP", "VPint_no_EB", "replacement", "regression_band"]
measures = ["MAE"]
significance_level = 0.05


for measure in measures:

    total_vals = {}
    for alg in algs:
        total_vals[pretty_names_methods[alg]] = np.array([])

    for scene in scenes:
        path = result_path + scene + "/"
        for feature in features:
            # First iterate over results per algorithm, store mean PM to be able to compute ranking
            alg_results = {}
            for alg in algs:
                path1 = path + alg + "_" + measure + ".csv"
                df = pd.read_csv(path1)
                df = df.drop_duplicates(subset=["patch"])
                # Drop patches where coverage was not complete (e.g., due to swath)
                invalid_patches_local = []
                invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
                invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
                df = df[~df['patch'].isin(invalid_patches_local)]
                # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
                # so will not affect performance to remove these.
                df = df[~df.patch.astype(str).str.isnumeric()] 
                vals = df[feature].values.astype(float)
                vals = vals[~np.isnan(vals)]
                if(CI_filter):
                    # Only keep results in specified confidence interval, since input data is not flawless
                    min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                    max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                    vals = vals[vals<max_val]
                    vals = vals[vals>min_val]

                if(len(vals) > 0):

                    if(measure == "SSIM" or measure == "PSNR"):
                        # To easily compute ranking for the same objective
                        # Autorank automatically does maximisation, so flipping the negative in this cell
                        pass
                    else:
                        vals = vals * -1

                    total_vals[pretty_names_methods[alg]] = np.concatenate((total_vals[pretty_names_methods[alg]], vals))

    for k,v in total_vals.items():
        if(len(v) != 13151):
            total_vals[k] = v[:13151] # TODO: remove
    
                    
    # Draw CD diagram
    df = pd.DataFrame(total_vals)
    result = autorank(df, alpha=significance_level, verbose=False)
    #print(result)

    #plot_stats(result)
    ar._util.cd_diagram(result, reverse=False, ax=None, width=6) # Same as result but allowing reverse (1 on left)
    plt.savefig("plots/CD_" + measure + "_patch.pdf", bbox_inches='tight')
    plt.show()


In [ ]:
# Visualise correlation between temporal distance and performance


pretty_names_methods = {
    "VPint":"VPint2",
    "VPint_no_IP":"VPint2 (no IP)",
    "VPint_no_EB":"VPint2 (no EB)",
    "regression":"AutoML regression",
    "regression_band":"AutoML regression",
    "replacement":"Replacement"
}

plot_kwargs_methods = {
    "VPint":{'color':'#377eb8', 'marker':'o'},
    "VPint_no_IP":{'color':'#e41a1c', 'marker':'v'},
    "VPint_no_EB":{'color':'#4daf4a', 'marker':'^'},
    "regression":{'color':'#984ea3', 'marker':'s'},
    "regression_band":{'color':'#984ea3', 'marker':'s'},
    "replacement":{'color':'#ff7f00', 'marker':'P'},
}

#features = [" 1m"]
algs = ["VPint", "VPint_no_IP", "VPint_no_EB", "replacement", "regression"]
algs = ["VPint", "regression", "replacement"]
measure = "MAE"

show_local = False


for alg in algs:
    feature_vals = {"1w":np.array([]), "1m":np.array([]), "3m":np.array([]), "6m":np.array([])}
    total_months = [0.23, 1, 3, 6]
    for scene in scenes:
        path = result_path + scene + "/"
        for feature in features:
            valset = []
            path1 = path + alg + "_" + measure + ".csv"
            df = pd.read_csv(path1)
            df = df.drop_duplicates(subset=["patch"])
            # Drop patches where coverage was not complete (e.g., due to swath)
            invalid_patches_local = []
            invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
            invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
            df = df[~df['patch'].isin(invalid_patches_local)]
            # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
            # so will not affect performance to remove these.
            df = df[~df.patch.astype(str).str.isnumeric()] 
            vals = df[feature].values
            if(CI_filter):
                # Only keep results in specified confidence interval, since input data is not flawless
                min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                vals = vals[vals<max_val]
                vals = vals[vals>min_val]

            #vals = vals[vals < 5000] # TODO: remove, this is simulating filtering
            if(len(vals) > 0):
                feature_vals[feature] = np.concatenate((feature_vals[feature], vals))

    feature_means = [np.nanmean(v) for k,v in feature_vals.items()]
    print(feature_means)
    plt.plot(total_months, feature_means, label=pretty_names_methods[alg], **plot_kwargs_methods[alg])

plt.xlabel("Temporal distance in months")
plt.ylabel("Median absolute error")
plt.title("Performance per temporal distance")
plt.xlim((0, 6.5))
plt.ylim((0, 1200))
plt.legend()
plt.savefig("plots/var_temporal_distance.pdf", bbox_inches='tight')
plt.show()


In [ ]:
# Visualise correlation between proportion of clouds and performance

import random
from scipy.interpolate import make_interp_spline, BSpline

with open("cloud_proportions.pkl", 'rb') as fp:
    cloud_percentages = pickle.load(fp)

pretty_names_methods = {
    "VPint":"VPint2",
    "VPint_no_IP":"VPint2 (no IP)",
    "VPint_no_EB":"VPint2 (no EB)",
    "regression":"AutoML regression",
    "regression_band":"AutoML regression",
    "replacement":"Replacement"
}

plot_kwargs_methods = {
    "VPint":{'color':'#377eb8'},#, 'marker':'o'},
    "VPint_no_IP":{'color':'#e41a1c'},#, 'marker':'v'},
    "VPint_no_EB":{'color':'#4daf4a'},#, 'marker':'^'},
    "regression":{'color':'#984ea3'},#, 'marker':'s'},
    "regression_band":{'color':'#984ea3'},#, 'marker':'s'},
    "replacement":{'color':'#ff7f00'},#, 'marker':'P'},
}

features = ["1w", "1m", "3m", "6m"] 
#features = [" 1m"]
algs = ["VPint", "VPint_no_IP", "VPint_no_EB", "replacement", "regression"]
algs = ["VPint", "regression", "replacement"]
measure = "MAE"
sample = False
sample_proportion = 0.005

yrange_max = 1200


for alg in algs:
    total_vals = np.array([])
    total_props = []
    for scene in scenes:
        path = result_path + scene + "/"
        for feature in features:
            valset = []
            path1 = path + alg + "_" + measure + ".csv"
            df = pd.read_csv(path1)
            df = df.drop_duplicates(subset=["patch"])
            # Drop patches where coverage was not complete (e.g., due to swath)
            invalid_patches_local = []
            invalid_patches_local.extend(invalid_patches[scene]['target']['coverage'])
            invalid_patches_local.extend(invalid_patches[scene][feature]['coverage'])
            df = df[~df['patch'].isin(invalid_patches_local)]
            # To get rid of i/o problems where writing was off (wrote pm as patch ID). Always affects duplicate rows
            # so will not affect performance to remove these.
            df = df[~df.patch.astype(str).str.isnumeric()] 
            df = df[df.patch.astype(str).str[0] == 'r'] 
            vals = df[feature].values
            patches = df['patch'].values
            if(CI_filter):
                # Only keep results in specified confidence interval, since input data is not flawless
                min_val = np.nanpercentile(vals, (100-CI_interval) / 2)
                max_val = np.nanpercentile(vals, 100 - (100-CI_interval) / 2)
                patches = patches[vals<max_val]
                vals = vals[vals<max_val]
                patches = patches[vals>min_val]
                vals = vals[vals>min_val]
                #vals = vals[vals < 100000] for testing

            if(len(vals) > 0):
                total_vals = np.concatenate((total_vals, vals))
                total_props.extend([int(cloud_percentages[scene][p]) for p in patches])

    if(sample):
        total_props, total_vals = zip(*random.sample(list(zip(total_props, total_vals)), int(sample_proportion * len(total_props))))

    
    xs = np.arange(start=0, stop=101) # stop is exclusive it seems
    #ys = np.zeros(101) # exclusive so 101
    ys = np.ones((101, len(total_vals))) * np.nan # Use matrix to compute median, std etc
    y_indices = np.zeros(len(total_vals)).astype(int) # index per x where to put new vals on axis=1 (because can't add+div)
    x_counts = np.zeros(101) # exclusive so 101
    for i in range(0, len(total_vals)):
        prop = int(total_props[i])
        val = total_vals[i]
        ys[prop, y_indices[prop]] = val
        x_counts[prop] += 1
        y_indices[prop] += 1
    xs = xs[x_counts>0]
    xs = xs[:-1]
    ys = ys[:-1]
    x_counts = x_counts[:-1]

    ys_median = np.nanmean(ys, axis=1) # using mean instead of median, don't want to change var names
    ys_std = np.nanstd(ys, axis=1) / 10
    ys_median = ys_median[x_counts>0]
    ys_std = ys_std[x_counts>0]
    #x_counts = x_counts[x_counts>0]
    #ys = ys / x_counts

    xnew = np.linspace(xs.min(), xs.max(), 20) 
    spl = make_interp_spline(xs, ys_median, k=3) 
    ys_median_smooth = spl(xnew)
    spl = make_interp_spline(xs, ys_std, k=3) 
    ys_std_smooth = spl(xnew)
    ys_std_smooth[ys_std_smooth<0] = 0
    ys_std_smooth[ys_std_smooth > yrange_max] = yrange_max

    plt.errorbar(xnew, ys_median_smooth, ys_std_smooth, label=pretty_names_methods[alg], 
                 capsize=5.0, errorevery=1, **plot_kwargs_methods[alg])
    #plt.plot(xnew, ys_smooth, label=pretty_names_methods[alg], **plot_kwargs_methods[alg])
    #plt.plot(xs, ys, label=pretty_names_methods[alg], **plot_kwargs_methods[alg])
        
    #plt.scatter(total_props, total_vals, label=pretty_names_methods[alg], **plot_kwargs_methods[alg])

plt.xlabel("Cloud cover percentage")
plt.ylabel("MAE")
plt.title("Mean MAE per cloud cover percentage, with STD")
#plt.xscale("log")
#plt.yscale("log")
plt.xlim((0, 100))
plt.ylim((0, yrange_max))
plt.legend()
plt.savefig("plots/var_cloud_percentage.pdf", bbox_inches='tight')
plt.show()


In [ ]:
# Computational efficiency plot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

base_path = "/mnt/c/Users/laure/Data/results/results_efficiency/"
save_path = "results/computational_efficiency.pdf"

methods = {
    "VPint": "VPint2",
    "VPint_multi": "VPint2 parallellised",
    "regression_band": "AutoML regression",
    "replacement": "Temporal replacement",
    "NSPI": "NSPI",
    "WLR": "WLR",
}

num_bins = 10
step_size = 100 / num_bins


for m, nice_name in methods.items():
    res_path = base_path + m + ".csv"
    df = pd.read_csv(res_path)

    means = []
    stds = []
    bins = []

    for b in range(0, num_bins):
        bin_min = step_size * b
        bin_max = step_size * (b+1)

        vals = df[df["clp"]>=bin_min]
        vals = df[df["clp"]<bin_max]
        vals = vals["running_time"].values
        mean = np.mean(vals)
        std = np.std(vals)
        means.append(mean)
        stds.append(std)
        bins.append((bin_max + bin_min) / 2)

    means = np.array(means)
    stds = np.array(stds)
    plt.plot(bins, means, label=nice_name)
    #plt.errorbar(bins, means, stds, label=nice_name)
    #plt.fill_between(bins, (means-stds), (means+stds), alpha=0.1)

plt.xlabel("Cloud cover percentage")
plt.ylabel("Running time in seconds (log scale)")
plt.yscale("log")
plt.title("Average running time per cloud cover percentage")
plt.legend()
plt.savefig(save_path, bbox_inches='tight')
plt.show()

    

In [ ]:
# SEN12MS-CR-TS results

import numpy as np
import pandas as pd

import os

metrics = ["RMSE", "SAM", "PSNR", "SSIM"]
methods = ["VPint", "VPint_multi"]
methods = ["VPint_multi"]
result_path = "/mnt/c/Users/laure/Downloads/grace/performance_measures_SEN12MS_CR_TS/"
result_path = "/mnt/c/Users/laure/Downloads/grace/performance_measures_SEN12MS_CR_TS_SeNSeI_WIP/"

for metric in metrics:
    print("\n")
    print(metric)

    for m in methods:
        print(m)
        results = np.array([], dtype=np.float32)
        for geo in os.listdir(result_path):
            local_path = result_path + geo + "/"
            for roi in os.listdir(local_path):
                local_path2 = local_path + roi + "/"
                for roi_num in os.listdir(local_path2):
                    local_path3 = local_path2 + roi_num + "/"
                    res_file = local_path3 + m + "_" + metric + ".csv"
                    df_vals = pd.read_csv(res_file)[metric].values
                    results = np.concatenate([results, df_vals])
        print(np.nanmean(results))


